# Retrieving datasets from QCFractal with `openff-qcsubmit`

**Based on the existing example**: https://github.com/openforcefield/openff-qcsubmit/blob/main/examples/retrieving-results.ipynb

This example shows how QCSubmit can be used to retrieve the results of quantum chemical (QC) calculations from a [QCFractal] instance such as [QCArchive].

In particular, it demonstrates how:

* raw torsion drive, optimised geometry and hessian result records can be retrieved from the public
  [QCArchive] server and stored in a result collection

* the retrieved result records can be filtered and curated using a set of built-in filters

* the result collection can be saved and loaded from disk

[QCFractal]: http://docs.qcarchive.molssi.org/projects/qcfractal/en/latest/
[QCArchive]: https://qcarchive.molssi.org/

For the sake of clarity all verbose warnings will be disabled in this tutorial:

In [ ]:
import warnings

warnings.filterwarnings("ignore")
import logging

logging.getLogger("openff.toolkit").setLevel(logging.ERROR)

## Retrieving result collections

QCSubmit provides a suite of utilities for retrieving and curating collections of QC results directly from a running QCFractal server, or an already computed QCPortal dataset. This functionality is provided through three main classes:

* `BasicResultCollection` - stores references to simple QCPortal result record that may contain energies, gradients, or hessians computed for a molecule in a single conformation.

* `OptimizationResultCollection` - stores references to full optimization result records (i.e. `OptimizationRecord`
  objects), as well as the final minimised conformer produced by the optimization.

* `TorsionDriveResultCollection` - stores references to full torsion drive result records (i.e. `TorsionDriveRecord`
  objects), as well as the minimum energy conformer associated with each torsion angle that was scanned.

Each of these collections can be generated directly from a running `QCFractal` server using the `from_server` class
method.

We begin by creating a QCPortal `FractalClient` instance that will allow us to communicate with the running
server. By default, `FractalClient` connects to the main QCArchive server:

In [ ]:
from qcportal import PortalClient

qc_client = PortalClient("https://api.qcarchive.molssi.org:443")

Other servers can be accessed by providing the server's URI.

We can then use this to generate our result collections:

In [ ]:
from openff.qcsubmit.results import (
    BasicResultCollection,
    OptimizationResultCollection,
    TorsionDriveResultCollection,
)

# Pull down the torsion drive records from the 'OpenFF Protein Capped 3-mer Backbones v1.0' dataset.
torsion_drive_result_collection = TorsionDriveResultCollection.from_server(
    client=qc_client,
    datasets="OpenFF Protein Capped 3-mer Backbones v1.0",
    spec_name="default",
)

*Note: currently only complete results are pulled down by the `from_server` method*

There are two main inputs to the `from_server` method, in addition to the fractal client:

* the name(s) of the existing datasets to retrieve the results of. This can either be the name of a single dataset or a list of dataset names
* the name of the specification used to compute the records. Each specification corresponds to a particular basis, method, program and additional settings.

Let's print out some basic information about each of these result collections:

In [ ]:
print("===TORSION DRIVE RESULTS===")

print(f"N RESULTS:   {torsion_drive_result_collection.n_results}")
print(f"N MOLECULES: {torsion_drive_result_collection.n_molecules}")

This allows results generated by multiple different servers (e.g. a local fractal instance and the public QCArchive
server) to be stored in a single result collection object.

The references to the actual data are then stored in corresponding lists:

In [ ]:
torsion_drive_result_collection.entries[qc_client.address][:10]

After running the above command, notice that the entries stored in the collection are not the actual result
records generated and stored on the server, but rather a reference to them. In particular, the unique ID of the record is stored along with a SMILES depiction of the molecule the result was generated for.

The main reason for doing this is that we often would like to be able to state which data we would like to use in
an application without having to create multiple copies of the data. Not only can this take up large amounts of disk space, it runs the risk of data becoming out of sync with the original if the format the records are stored in changes or the local copy of the data is accidentally mutated. Storing a reference to the original data and then retrieving it when needed is typically a cleaner and safer solution.

## Retrieving the result records

The raw result record objects can be easily retrieved using the result collection objects. This allows us to filter the collection to only retrieve the results we want. For example, we can apply a SMARTS string to retrieve the cysteine record:

In [ ]:
from openff.qcsubmit.results.filters import SMARTSFilter

filtered_collection = torsion_drive_result_collection.filter(SMARTSFilter(smarts_to_include=["C[SH]"]))
filtered_collection

Then we can download the actual results. This can be very slow, so it's worth filtering aggressively:

In [ ]:
torsion_drive_records = filtered_collection.to_records()
torsion_drive_records

QCSubmit seamlessly takes care of pulling the data from the server in the most efficient way making sure to take
advantage of the pagination that QCFractal provides. Further, it attempts to cache all calls to the server so that
multiple calls to `to_records` does not need to constantly query the server.

Notice that not only are the raw result records retrieved, but also an OpenFF `Molecule` object is created for each result record. This molecule has the correct ordering and also stores any conformers associated with the
result collection. For basic collections, the conformer is the one that was used in any calculations; for optimization collections, it is the final conformer yielded by the optimization; and for torsion drives, it is the lowest energy conformer for each sampled torsion angle.

## Inspecting results

In the case of torsion drive records, we can easily iterate over the grid ID, the associated conformer, and the
associated energy in one go:

In [ ]:
torsion_drive_record, molecule = torsion_drive_records[0]
import numpy as np
from matplotlib import pyplot
from openff.units import unit

energy_grid = np.zeros((24, 24))
psi_labels = [""] * 24
phi_labels = [""] * 24
for (phi, psi), qc_conformer in zip(
    molecule.properties["grid_ids"], molecule.conformers
):
    qc_energy = torsion_drive_record.final_energies[(phi, psi)]

    phi_bin = int(phi + 165) // 15
    psi_bin = int(psi + 165) // 15
    energy_grid[psi_bin, phi_bin] = qc_energy
    psi_labels[psi_bin] = psi
    phi_labels[phi_bin] = phi
    # print(f"({phi}, {psi}) {phi_bin},{psi_bin} E={qc_energy:.4f} Ha")

In [ ]:
import plotly.graph_objects as go
from ipywidgets import widgets

fig = go.FigureWidget(
    data=go.Heatmap(
        z=energy_grid,
        x=phi_labels,
        y=psi_labels,
        colorbar={"title": "Energy (Ha)"},
        hovertemplate="phi: %{x}\npsi: %{y}\nenergy: %{z} Ha",
    ),
    layout=go.Layout(
        title="Val-Ala-Val - central backbone torsiondrive (Ha)",
        xaxis_title="Phi",
        yaxis_title="Psi",
        # autosize=False,
        yaxis_scaleanchor="x",
        xaxis_scaleanchor="y",
    ),
)

view = molecule.visualize("nglview")

def on_click(trace, points, selector):
    print(points)
    for x, y in points.point_inds:
        view.frame = x * 24 + y


heatmap = fig.data[0]
heatmap.on_click(on_click)

container = widgets.GridBox(
    [fig, view],
    # layout=widgets.Layout(grid_template_columns=f"repeat(2, {600}px)"),
)
container